# 추천 시스템

### 4. 잠재 요인 협업 필터링

#### 확률적 경사 하강법을 이용한 행렬 분해

In [1]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재요인 차원 K는 3으로 설정
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 2, np.NaN]])
num_users, num_items = R.shape  # R: 사용자-아이템 행렬
K=3

# P와 Q 행렬 크기를 지정하고 정규분포를 가진 임의의 값으로 입력
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

* 실제 R 행렬과 예측 행렬의 오차 구하기

In [2]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]  # 실제 R 행렬에서 널 아닌 값
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]  # 예측 행렬에서 해당 위치 인덱스 값

    # 실제 R 행렬과 예측 행렬의 RMSE 추출
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

* SGD 기반 행렬 분해

In [3]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
non_zeros = [(i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0]

# steps(반복 업데이트 횟수), learning_rate(학습률), r_lambda(L2 규제 계수) 설정
steps=1000
learning_rate=0.01
r_lambda=0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij*Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij*P[i, :] - r_lambda*Q[j,:])

    # 50회 반복할 때마다 오류 값 출력
    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print("### iteration step : ", step," rmse : ", rmse)

### iteration step :  0  rmse :  3.2388050277987723
### iteration step :  50  rmse :  0.4876723101369648
### iteration step :  100  rmse :  0.1564340384819247
### iteration step :  150  rmse :  0.07455141311978046
### iteration step :  200  rmse :  0.04325226798579314
### iteration step :  250  rmse :  0.029248328780878973
### iteration step :  300  rmse :  0.022621116143829466
### iteration step :  350  rmse :  0.019493636196525135
### iteration step :  400  rmse :  0.018022719092132704
### iteration step :  450  rmse :  0.01731968595344266
### iteration step :  500  rmse :  0.016973657887570753
### iteration step :  550  rmse :  0.016796804595895633
### iteration step :  600  rmse :  0.01670132290188466
### iteration step :  650  rmse :  0.01664473691247669
### iteration step :  700  rmse :  0.016605910068210026
### iteration step :  750  rmse :  0.016574200475705
### iteration step :  800  rmse :  0.01654431582921597
### iteration step :  850  rmse :  0.01651375177473524
### iterati

* 분해된 P, Q 행렬로 예측 행렬 만들기

In [4]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


In [5]:
print(R)

[[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]


In [ ]:
# 원본 행렬과 비교할 때, 널 아닌 값은 큰 차이 나지 않음

### 8. 파이썬 추천 시스템 패키지 - Surprise

#### Surprise 패키지소개

In [7]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357272 sha256=13c663e448acad2984b305bb923a9455c7497310465bd536356f41fab3d6c0ab
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


#### Surprise 이용한 추천 시스템 구축

In [8]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [9]:
data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


* 알고리즘 객체 생성

In [10]:
algo = SVD()

# 학습 데이터 기반 추천 알고리즘 학습
algo.fit(trainset)

##### test( ) 메서드

In [11]:
# test()로 검증 데이터에 대해 추천 수행
predictions = algo.test(testset)
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5420490217434244, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=4.064088825693579, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.906510091281124, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.567075899332699, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.257356138581865, details={'was_impossible': False})]

In [12]:
# Prediction 객체
# : 개별 사용자 아이디(uid), 영화 아이디(iid), 평점(r_ui) 정보 기반 => 추천 예측 평점(est) 데이터
# : details: 추천 예측 할 수 없을 때 로그용으로 데이터 남김 (was_impossible=True: 예측값 생성할 수 없음)

In [13]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5420490217434244),
 ('882', '291', 4.064088825693579),
 ('535', '507', 3.906510091281124)]

##### predict( ) 메서드

* predict( ): 개별 사용자 아이템에 대한 추천 평점 예측
* test( ): 모든 사용자, 아이템 아이디에 대해 predict( )를 반복적으로 수행

In [14]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 3.99   {'was_impossible': False}


In [15]:
# 추천 시스템 평가 정보
accuracy.rmse(predictions)

RMSE: 0.9509


0.9508991861025147

#### Surprise 주요 모듈 소개

##### OS 파일 데이터를 Surprise 데이터 세트로 로딩


In [20]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/gdrive/')

path="/content/gdrive/MyDrive/Euron/ml-latest-small/"

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [21]:
import pandas as pd

ratings = pd.read_csv(path + 'ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 언로드시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv(path + 'ml-latest-small/ratings_noh.csv', index=False, header=False)

In [23]:
from surprise import Reader

# 4개의 칼럼 (user, item, rating, timestamp)으로 구성되어 있다고 명시
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))  # 칼럼 분리 문자: ',', 평점: 단위 0.5, 최대 5
data=Dataset.load_from_file(path + '/ml-latest-small/ratings_noh.csv',reader=reader)  # 로딩 시 앞 3개의 칼럼만 로딩됨

* SVD 기법으로 추천 예측

In [24]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)  # n_factor: 잠재 요인 크기 K 값

# 학습 데이터 세트로 학습
algo.fit(trainset)
# 테스트 데이터 세트로 평점 예측
predictions = algo.test(testset)
# RMSE 평가
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

##### 판다스 DataFrame에서 Surprise 데이터 세트로 로딩

In [25]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv(path + 'ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5,5.0))

# ratings DataFrame에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 함
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 추천 알고리즘 객체 생성
algo = SVD(n_factors=50, random_state=0)
# 학습 데이터로 학습
algo.fit(trainset)
# 테스트 데이터 세트로 평점 예측
predictions = algo.test(testset)
# RMSE 평가
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### 교차 검증과 하이퍼 파라미터 튜닝

In [26]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv(path + '/ml-latest-small/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
# 교차 검증 후 RMSE, MAE로 성능 평가
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8707  0.8845  0.8686  0.8645  0.8736  0.8724  0.0067  
MAE (testset)     0.6698  0.6809  0.6662  0.6643  0.6727  0.6708  0.0058  
Fit time          1.47    1.49    1.47    1.43    2.26    1.63    0.32    
Test time         0.26    0.11    0.11    0.11    0.19    0.15    0.06    


{'test_rmse': array([0.87067389, 0.88447785, 0.8686039 , 0.86453146, 0.87360213]),
 'test_mae': array([0.66975423, 0.68087162, 0.66615616, 0.66429301, 0.67269149]),
 'fit_time': (1.4679129123687744,
  1.4937829971313477,
  1.4719772338867188,
  1.4312539100646973,
  2.2610397338867188),
 'test_time': (0.2627103328704834,
  0.1101217269897461,
  0.10690641403198242,
  0.10848450660705566,
  0.1850743293762207)}

* 하이퍼 파라미터 최적화

In [27]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8766257301024302
{'n_epochs': 20, 'n_factors': 50}


#### Surprise를 이용한 개인화 영화 추천 시스템 구축

In [28]:
# 다음 코드는 train_test_split( )으로 분리되지 않는 데이터 세트에 fit( )을 호출해 오류가 발생함
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'n_users'

* 전체 데이터를 학습 데이터로 사용 (DatasetAutoFolds)

In [29]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file= path + '/ml-latest-small/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습데이터로 생성
trainset = data_folds.build_full_trainset()

In [30]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
# 학습 데이터 기반 학습 수행
algo.fit(trainset)

In [31]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv(path + '/ml-latest-small/movies.csv')

# userId=9 의 movieId 데이터 추출
movieIds = ratings[ratings['userId']==9]['movieId']
# movieId=42 데이터가 있는지 확인
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [32]:
uid = str(9)
iid = str(42)

# 추천 예측 평점
pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


* 추천 대상이 되는 영화 추출

In [33]:
def get_unseen_surprise(ratings, movies, userId):
    # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()

    # 모든 영화들의 movieId를 리스트로 생성.
    total_movies = movies['movieId'].tolist()

    # 모든 영화들의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천 대상 영화수:',len(unseen_movies), '전체 영화수:',len(total_movies))

    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천 대상 영화수: 9696 전체 영화수: 9742


* 높은 예측 평점 가진 순으로 영화 추천

In [34]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):

    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]

    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행
    def sortkey_est(pred):
        return pred.est

    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행
    predictions.sort(key=sortkey_est, reverse=True)
    # top_n개의 최상위 값 추출
    top_predictions= predictions[:top_n]

    # top_n으로 추출된 영화의 정보 추출. (영화 아이디, 추천 예상 평점, 제목 추출)
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

    top_movie_preds = [(id, title, rating) for id, title, rating in \
                       zip(top_movie_ids, top_movie_titles, top_movie_rating)]

    return top_movie_preds

# userId: 9 추천 영화 추출
unseen_movies = get_unseen_surprise(ratings, movies, 9)
# 높은 평점 순(TOP 10)으로 영화 추천
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천 대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122657
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
